In [2]:
from scipy import stats 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Epydemix import
import sys
sys.path.append("../")
from epydemix.population import Population
from epydemix.epimodel import EpiModel, stochastic_simulation
from epydemix.plotting import plot_quantiles, plot_selected_quantiles, plot_posterior, plot_posterior_2d
from epydemix.calibration import calibration_top_perc

# EpiModel Object

The ```EpiModel``` object is the core of the ```epydemix``` simulation module. With ```EpiModel``` objects we can define flexible epidemic model and then simulate them. Before creating the first ```EpiModel``` we need to create a ```Population``` object, which is needed for the simulation

In [3]:
population = Population() 
population.load_population(population_name="Indonesia", path_to_data="../epydemix_data/")

We can now define the ```EpiModel``` object. This requires three simple steps: 
- Definition of compartments
- Definition of model parameters
- Definition of transitions


Parameters can also be defined before compartments, while transitions must be defined after the definition of compartments, otherwise this will raise an error. Let's see how this works in practive by creating a simple SIR model:

In [5]:
# create model 
model = EpiModel(compartments=["S", "R"])

# compartments can be also added after the first init call
model.add_compartments("I")

# define parameters
model.add_parameters({"bet": 0.04, "mu": 0.1})

# add transitions 
model.add_transition(source="S", target="I", rate_name="bet", agent="I")
model.add_transition(source="I", target="R", rate_name="mu")

In more details: 
- compartments are string identifying the name of the compartment. They can be added when creating the ```EpiModel``` object of afterwards using the ```add_compartments``` method (which takes as input both single and list of compartments)

- parameters are represented as a dictionary whose keys (values) are the names (values) of the parameters. They can be added when creating the ```EpiModel``` object of afterwards using the ```add_parameters``` method, which takes as input a dictionary. The dictionary can contain single or multiple parameters. 

- transitions are represented as ```Transition``` objects and they have the following attributes: 
    - source: the name of the source compartment of the transition
    - target: the name of the target compartment of the transition
    - rate_name: the expression of the rate of the transition
    - agent: the name of the compartment which catalyzes the transition (in case of spontaneous transitions it is set to None)

Important Note: some parameter names, such as ```beta``` and ```gamma``` cannot be used. This is because ```rate_name``` are evaluated as composite expressions using ```sympy```, and some names may be in conflict with this python package. This is done to allow for transitions that have composite expressions for the rate (more details below).